# Compare Inventory ananlysis results in different period
1. Find diffence bet two distribution reports of store 108.
2. Check robust. (How many common SKU are there). 

### Output 
path `../data/Output_Sep_108/fullMonthAllPeriodSKU.csv`

#### dataset required before run this notebook: 
- `Output_Sep_108`: **Rename** output folder of 108 store (Sep-Nov)
- `Output_July_108`: **Rename** output folder of 108 store (July-Sep)


## Questions:
### Distrubution report
1. How many (%) SKU has different facing or capacity in these two period?
   - 125/679 = **18%**
1. what reason leads to adjust in distribution report?

### Full Month SKU
1. The percentage of SKUs (full month), in both period? 
   - 74/110 = **67%**
2. **Does fullmonth SKU (based on July-Sep) capacity decrease in new distribution report?**
   - There are 55 SKU's capacity decrease from July to Sep, 
      1. only 8 of them are shown in fullMonthSKU in July
      2. 7 of them are shown in fullMonthSKU in two period.
   - (`Incorrect decision`): There are 68 SKU's capacity increase from July to Sep, 
      1. 17 of them are shown in fullMonthSKU in July. 
      2. 8 of them are shown in fullMonthSKU in two period.
   - Thus, the current Capacity adjustment decision has no strong relationship with Qtysold of previous month.

3. **For the fullmonth SKU, which in one period only, does them appear in atLeastoneMonthSKU of another period?**

### Issued SKU

In [55]:
import pandas as pd
from pandas import ExcelWriter
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from Inventory_opti_helperFunction import *

In [56]:
# Python code t get difference of two lists 
# Using set() 
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 
  
# Driver Code 
li1 = [10, 15, 20, 25, 30, 35, 40] 
li2 = [25, 40, 35, 34343] 
print(Diff(li1, li2)) 

[10, 20, 30, 15]


### Compare two distribution report

In [57]:
dist_df_Jul  = pd.read_csv("../data/RawData/Distribution Report - Auckland Departures - July19.csv")
dist_df_Sep  = pd.read_csv("../data/RawData/Distribution Report- Auckland Departures - Jan2020.csv")
dist_df_Sep = dist_df_Sep[['Mat ID', 'Name', '# POGs', 'Facings', 'Capacity']]
dist_df_Jul = dist_df_Jul[['Mat ID', 'Name', '# POGs', 'Facings', 'Capacity']]

In [58]:
125/679

0.18409425625920472

In [59]:
dist_df = pd.merge(dist_df_Jul, dist_df_Sep, on=['Mat ID', 'Name'])
dist_df_adjust = dist_df[(dist_df['Capacity_x']!=dist_df['Capacity_y']) | (dist_df['Facings_x']!=dist_df['Facings_y'])]

### Compare full month SKU

In [60]:
df_SepNov  = pd.read_csv("../data/Output_Sep_108/fullMonth/qty_less_than_capacity_all_month_SKU.csv")
df_JulSep  = pd.read_csv("../data/Output_July_108/fullMonth/qty_less_than_capacity_all_month_SKU.csv")

In [61]:
SKU_SepNov  = df_SepNov['SKU'].unique()
SKU_JulSep  = df_JulSep['SKU'].unique()

print("{} Unique SKU (fullmonth) in SepNov".format(len(SKU_SepNov)))
print("{} Unique SKU (fullmonth) in JulSep".format(len(SKU_JulSep)))

110 Unique SKU (fullmonth) in SepNov
117 Unique SKU (fullmonth) in JulSep


In [62]:
# Python code t get difference of two lists 
# Using set() 
def Diff(li1: list, li2:list) -> list:
    """
    get the different item shown in li1, not li2
    """
    return (list(set(li1) - set(li2))) 

def test_SKU_has_6_obs(result):
    """
    test each SKU has 6 observations
    """
    lst = list(result.groupby('SKU')['MatID'].count())
    assert len(set(lst)) == 1
    assert lst[0] ==6

#### Common SKU in two period

In [63]:
SKU_common = list(set(SKU_SepNov).intersection(SKU_JulSep))
print("{} Unique SKU (fullmonth) in SepNov and JulSep".format(len(SKU_common)))

74 Unique SKU (fullmonth) in SepNov and JulSep


In [64]:
def save_fullMonthAllPeriodSKU(SKU_common: list, df_SepNov: pandas.core.frame.DataFrame, 
                              df_JulSep: pandas.core.frame.DataFrame):
    """
    Save the common SKU, which full month in two period to CSV
    """

    SKU_common_toDF = pd.DataFrame(SKU_common, columns = ['SKU'])
    result = pd.concat([df_SepNov, df_JulSep]).merge(SKU_common_toDF, on = ['SKU'], how = 'inner') # select 74 common items 
    # test each item show 6 time in dataset
    test_SKU_has_6_obs(result)
    
    # add StdQty_to_AvgQty column
    # SepNov
    AllatLeastOneMonth_SKU  = pd.read_csv("../data/Output_Sep_108/atLeastOneMonth/AllatLeastOneMonth_SKU.csv")
    AllatLeastOneMonth_SKU = AllatLeastOneMonth_SKU[['MatID', 
                                                     'StdQty_to_AvgQty']].rename(columns={"StdQty_to_AvgQty": "StdQty_to_AvgQty_SepNov"})
    SKU_common_df = pd.merge(AllatLeastOneMonth_SKU, result, on = ['MatID'], how='right')
    
    # JulSep
    AllatLeastOneMonth_SKU  = pd.read_csv("../data/Output_July_108/atLeastOneMonth/AllatLeastOneMonth_SKU.csv")
    AllatLeastOneMonth_SKU = AllatLeastOneMonth_SKU[['MatID', 
                                                     'StdQty_to_AvgQty']].rename(columns={"StdQty_to_AvgQty": "StdQty_to_AvgQty_JulSep"})
    SKU_common_df = pd.merge(AllatLeastOneMonth_SKU, SKU_common_df, on = ['MatID'], how='right')
    
    SKU_common_df.to_csv('../data/Output_Sep_108/fullMonthAllPeriodSKU.csv', 
                                     index=False, encoding='utf-8')
    

In [65]:
save_fullMonthAllPeriodSKU(SKU_common, df_SepNov, df_JulSep)

#### Difference bet two period
check if these items 

In [66]:
SKU_common_df = pd.DataFrame(SKU_common , columns = ['Name']) 

In [68]:
SKU_inSepNot_notJulSep = Diff(SKU_SepNov, SKU_JulSep)

In [69]:
SKU_inJulSep_notSepNov = Diff(SKU_JulSep, SKU_SepNov)

### Question: Does fullmonth SKU (based on July-Sep) capacity decrease in new distribution report?

There are 55 SKU's capacity decrease from July to Sep, but only 8 of them are shown in fullMonthSKU in July.

In [70]:
dist_df_capacity_decrease = dist_df[(dist_df['Capacity_x'] > dist_df['Capacity_y'])]
print(len(dist_df_capacity_decrease))

# check items which are fullMonthS SKU in July
check_fullMonth_july = pd.merge(dist_df_capacity_decrease, df_JulSep, left_on = ['Mat ID', 'Name'], 
                 right_on=['MatID', 'SKU'], how='inner')
print(len(check_fullMonth_july['MatID'].unique())) # number of items find 

# check items which are fullMonthS SKU in 2 period
check_common =  pd.merge(dist_df_capacity_decrease, SKU_common_df, on = 'Name', how='inner')
print(len(check_common['Name'].unique())) # number of items find

68
17
8


In [71]:
dist_df_capacity_increase = dist_df[(dist_df['Capacity_x'] < dist_df['Capacity_y'])]
print(len(dist_df_capacity_increase))

check = pd.merge(dist_df_capacity_increase, df_JulSep, left_on = ['Mat ID', 'Name'], 
                 right_on=['MatID', 'SKU'], how='inner')
print(len(check['MatID'].unique())) # number of items find 

# check items which are fullMonthS SKU in 2 period
check_common =  pd.merge(dist_df_capacity_increase, SKU_common_df, on = 'Name', how='inner')
print(len(check_common['Name'].unique())) # number of items find

55
8
7
